# Modeling – Baseline Churn Prediction

## MLflow Configuration

In [1]:
from pathlib import Path
import mlflow

PROJECT_ROOT = Path.cwd().parent.resolve()  # notebook is inside "Note Books"

mlflow.set_tracking_uri(f"file:{(PROJECT_ROOT/ 'mlruns').as_posix()}")
mlflow.set_experiment("customer_churn_prediction")

print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: file:E:/My Projects/Customer Churn Prediction with Automated ML Pipeline/mlruns


c:\Users\HP\anaconda3\envs\churn-env\lib\site-packages\mlflow\tracking\_tracking_service\utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)


## Import required libraries

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
import mlflow.sklearn

## Load Data & Basic Cleaning

In [3]:
df = pd.read_csv("../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Fix TotalCharges
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

# Encode target
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Drop ID column
df.drop('customerID', axis=1, inplace=True)

## Separate features and target

In [4]:
X = df.drop('Churn', axis=1)
y = df['Churn']

## Feature Groups

In [5]:
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
cat_cols = [col for col in X.columns if col not in num_cols]

## Split the data set

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,stratify=y,random_state=42)

## Preprocessing Pipeline

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ]
)

## Baseline Model Pipeline

In [8]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

## Model Training & Evaluation

In [9]:
with mlflow.start_run(run_name="Logistic Regression - Baseline"):

    mlflow.log_param("model", "logistic_regression")
    mlflow.log_param("class_weight", "none")

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    report = classification_report(y_test, y_pred, output_dict=True)

    mlflow.log_metric("accuracy", report["accuracy"])
    mlflow.log_metric("recall_churn", report["1"]["recall"])
    mlflow.log_metric("precision_churn", report["1"]["precision"])
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_prob))

    mlflow.sklearn.log_model(model, "model")


2026/01/28 08:11:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


## Baseline Model Interpretation

A Logistic Regression model was used as the baseline for churn prediction.
Model performance was tracked using MLflow to enable consistent comparison across experiments.

- The model performs well in identifying non-churn customers, achieving high recall and precision for class 0.
- Performance on churn customers (class = 1) is weaker, with recall significantly lower than non-churn, indicating that a high portion of churners are not detected.
- This behavior is expected due to class imbalance in the dataset
- Accuracy and weighted-average metrics appear strong but are influenced by the majority class and therefore are not reliable indicators of churn detection quality.
- Recall for churn customers is identified as the primary metric for business evaluation, as failing to identify churners results in direct customer loss.

This baseline establishes a reference point for evaluating subsequent model improvements.

## Model Improvement Strategy

Based on baseline performance, recall for churn customers requires improvement.  
Planned improvements focus on:
1. Addressing class imbalance using class weighting. 
2. Optimizing the decision threshold to align with business objectives.
3. Evaluating more expressive models (tree-based).


## Model Improvement 1 Class-Balanced Logistic Regression

In [10]:
weighted_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        random_state=42
    ))
])


## Weighted Model Training & Evaluation

In [11]:
with mlflow.start_run(run_name="Logistic Regression - Weighted"):

    mlflow.log_param("model", "logistic_regression")
    mlflow.log_param("class_weight", "balanced")

    weighted_model.fit(X_train, y_train)

    y_pred = weighted_model.predict(X_test)
    y_prob = weighted_model.predict_proba(X_test)[:, 1]

    report = classification_report(y_test, y_pred, output_dict=True)

    mlflow.log_metric("accuracy", report["accuracy"])
    mlflow.log_metric("recall_churn", report["1"]["recall"])
    mlflow.log_metric("precision_churn", report["1"]["precision"])
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_prob))

    mlflow.sklearn.log_model(weighted_model, "model")


2026/01/28 08:11:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


## Weighted Model Interpretation

Applying class weighting significantly changes the model’s behavior by prioritizing churn customers.

- Recall for churn customers (class = 1) improves substantially, indicating the model now captures a much larger proportion of customers who are likely to churn.
- This improvement comes at the cost of reduced recall for non-churn customers and a lower overall accuracy, reflecting an increase in false positive churn predictions.
- Precision for churn decreases, meaning more customers are incorrectly flagged as potential churners; however, this trade-off is acceptable in a churn prevention context where missing churners is more costly than offering retention incentives to non-churners.
- Macro-averaged metrics improve compared to the baseline, confirming a more balanced performance across classes.
- Overall, the class-weighted Logistic Regression aligns better with business objectives by reducing false negatives for churn, making it a more suitable model than the baseline despite lower accuracy.


## Model Improvement - Decision Threshold Optimization

### Get predicted probabilities

In [12]:
y_proba = weighted_model.predict_proba(X_test)[:,1]

### Evaluate multiple thresholds

In [13]:
from sklearn.metrics import precision_score, recall_score

thresholds = [0.3, 0.4, 0.5, 0.6]
for t in thresholds:
    y_pred_t = (y_proba >= t).astype(int)
    precision = precision_score(y_test, y_pred_t, pos_label=1)
    recall = recall_score(y_test, y_pred_t, pos_label=1)
    print (f"Threshold: {t:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f}")

Threshold: 0.30 | Precision: 0.43 | Recall: 0.93
Threshold: 0.40 | Precision: 0.47 | Recall: 0.87
Threshold: 0.50 | Precision: 0.50 | Recall: 0.78
Threshold: 0.60 | Precision: 0.54 | Recall: 0.71


### Final evaluation with chosen threshold

In [14]:
best_threshold = 0.4

y_pred_best = (y_proba >= best_threshold).astype(int)

print(confusion_matrix(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

[[663 372]
 [ 50 324]]
              precision    recall  f1-score   support

           0       0.93      0.64      0.76      1035
           1       0.47      0.87      0.61       374

    accuracy                           0.70      1409
   macro avg       0.70      0.75      0.68      1409
weighted avg       0.81      0.70      0.72      1409



## Threshold Optimization Interpretation

Adjusting the decision threshold allows control over the precision–recall trade-off without retraining the model.

- Lowering the threshold increases recall for churn customers by classifying more customers as potential churners.
- This results in a higher number of false positives, reducing precision, which is acceptable in a churn prevention context.
- The selected threshold improves the model’s ability to identify churners compared to the default 0.5 threshold.
- Threshold optimization further aligns the model with business objectives by prioritizing churn detection over overall accuracy.
- This threshold balances business risk and operational cost more effectively than more aggressive or conservative thresholds.


## Model Improvement Tree-Based Model (Random Forest)

### Validation Strategy

To avoid test leakage, model selection and tuning are performed using cross-validation
on the training set only. The test set is reserved for final evaluation.

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

## Build Random Forest Pipeline

In [16]:
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        class_weight='balanced'
    ))
])

## Random Forest – Cross-Validation

In [22]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_recall = cross_val_score(rf_model,X_train,y_train,cv=cv,scoring='recall')

print("CV Recall scores:", cv_recall)
print("Mean CV Recall:", cv_recall.mean())

CV Recall scores: [0.46822742 0.44147157 0.51505017 0.4548495  0.46153846]
Mean CV Recall: 0.46822742474916385


### Random Forest Cross-Validation Interpretation

Random Forest was evaluated using stratified cross-validation on the training set.
The model shows stable performance across folds; however, it does not demonstrate
an improvement in recall for churn customers relative to the previously evaluated
logistic regression models.

This indicates that the churn-related patterns in the dataset are well captured
by linear decision boundaries and that increased model complexity does not provide
additional benefit for the primary business objective.
Given the importance of maximizing churn recall, tree-based models were not selected
for further optimization.

## Random Forest – MLflow Evaluation

In [18]:
with mlflow.start_run(run_name="Random Forest - Balanced"):

    mlflow.log_param("model", "random_forest")
    mlflow.log_param("n_estimators", 200)
    mlflow.log_param("class_weight", "balanced")

    rf_model.fit(X_train, y_train)

    y_pred = rf_model.predict(X_test)
    y_prob = rf_model.predict_proba(X_test)[:, 1]

    report = classification_report(y_test, y_pred, output_dict=True)

    mlflow.log_metric("accuracy", report["accuracy"])
    mlflow.log_metric("recall_churn", report["1"]["recall"])
    mlflow.log_metric("precision_churn", report["1"]["precision"])
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_prob))

    mlflow.sklearn.log_model(rf_model, "model")

2026/01/28 08:12:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


## Final Model Selection

Based on MLflow-tracked test set performance, cross-validation analysis,
and business objectives, the class-weighted Logistic Regression model
with an optimized decision threshold is selected as the final model.

While Random Forest exhibits stable behavior during cross-validation,
it does not provide a meaningful improvement in churn recall compared
to the linear model.
Considering the priority of churn detection, model interpretability,
and deployment simplicity, Logistic Regression is the preferred choice.

## FINAL model

In [19]:
final_model = weighted_model

## Final Test Evaluation

In [20]:
y_test_proba = final_model.predict_proba(X_test)[:, 1]

final_threshold = 0.4
y_test_pred = (y_test_proba >= final_threshold).astype(int)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[663 372]
 [ 50 324]]
              precision    recall  f1-score   support

           0       0.93      0.64      0.76      1035
           1       0.47      0.87      0.61       374

    accuracy                           0.70      1409
   macro avg       0.70      0.75      0.68      1409
weighted avg       0.81      0.70      0.72      1409



## Final Model Evaluation

The final model was evaluated once on the held-out test set.

- The model achieves high recall for churn customers, successfully identifying
  the majority of customers at risk of leaving.
- Precision is lower than recall, indicating an increased number of false positives,
  which is acceptable in a churn prevention scenario.
- Overall accuracy is lower than baseline models but does not reflect business value.
- The selected decision threshold reflects the asymmetric cost of churn prediction.

This model provides a strong balance between performance, interpretability,
and business alignment.

## Project Conclusion

This project developed an end-to-end churn prediction pipeline using
structured customer data.

Key takeaways:
- Logistic Regression provided strong baseline performance and interpretability.
- Class imbalance significantly affected churn recall and required mitigation.
- Threshold optimization proved critical for aligning predictions with business costs.
- More complex models did not outperform simpler, well-tuned linear models.

The final solution demonstrates the importance of disciplined model evaluation
and business-driven decision making in applied machine learning.

## Save the final model

In [23]:
import joblib

MODEL_DIR = Path("../Models")
MODEL_DIR.mkdir(exist_ok=True)


joblib.dump(final_model, MODEL_DIR / "final_churn_model.pkl")

['..\\Models\\final_churn_model.pkl']